<a href="https://colab.research.google.com/github/jjlizcano/Sistema-recomendaciones-cultivo/blob/main/notebooks/Primera_entrega_proyecto_IA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# 1. Instalar dependencias adicionales
!pip install gdown seaborn --quiet

# 2. Clonar o actualizar el repositorio
import os

REPO_URL = "https://github.com/jjlizcano/Sistema-recomendaciones-cultivo.git"
REPO_DIR = "Sistema-recomendaciones-cultivo"

if not os.path.exists(REPO_DIR):
    !git clone {REPO_URL}
else:
    %cd {REPO_DIR}
    !git pull
    %cd ..

print("Repositorio listo en:", REPO_DIR)


Cloning into 'Sistema-recomendaciones-cultivo'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 15 (delta 3), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (15/15), 345.18 KiB | 2.08 MiB/s, done.
Resolving deltas: 100% (3/3), done.
Repositorio listo en: Sistema-recomendaciones-cultivo


In [4]:
# 3. Semilla reproducible
import numpy as np, random

SEED = 42
np.random.seed(SEED)
random.seed(SEED)

In [5]:
# 4. Verificar entorno
import sys, pandas as pd

print("Python:", sys.version)
print("Pandas:", pd.__version__)
print("Numpy:", np.__version__)


Python: 3.12.11 (main, Jun  4 2025, 08:56:18) [GCC 11.4.0]
Pandas: 2.2.2
Numpy: 2.0.2


In [7]:
# 5. Cargar dataset desde /data
csv_path = f"{REPO_DIR}/data/FAOSTAT_data_en_9-1-2025.csv"

try:
    df = pd.read_csv(csv_path)
    print("Dataset cargado correctamente")
    print("Registros en el dataset:", len(df))
    display(df.head())
except FileNotFoundError:
    print(f" Error: No se encontró el archivo {csv_path}")

Dataset cargado correctamente
Registros en el dataset: 29272


,Domain Code,Domain,Area Code (M49),Area,Element Code,Element,Item Code (CPC),Item,Year Code,Year,Unit,Value,Flag,Flag Description,Note
0,QCL,Crops and livestock products,4,Afghanistan,5312,Area harvested,1371.0,"Almonds, in shell",2023,2023,ha,37000.0,A,Official figure,NaN
1,QCL,Crops and livestock products,4,Afghanistan,5412,Yield,1371.0,"Almonds, in shell",2023,2023,kg/ha,1810.8,A,Official figure,NaN
2,QCL,Crops and livestock products,4,Afghanistan,5510,Production,1371.0,"Almonds, in shell",2023,2023,t,67000.0,A,Official figure,NaN
3,QCL,Crops and livestock products,4,Afghanistan,5312,Area harvested,1654.0,"Anise, badian, coriander, cumin, caraway, fenn...",2023,2023,ha,25439.0,E,Estimated value,NaN
4,QCL,Crops and livestock products,4,Afghanistan,5412,Yield,1654.0,"Anise, badian, coriander, cumin, caraway, fenn...",2023,2023,kg/ha,704.0,E,Estimated value,NaN
